# SyriaTel Customer Churn Project

## Overview
This project seeks to predict customer churn for SyriaTel using historical usage and customer service data. By leveraging classification models, we aim to detect patterns that distinguish churners from loyal customers. The end goal is to empower the business with proactive strategies to reduce churn and retain high-value customers. The full machine learning lifecycle is followed—from problem framing and data exploration to model development and business recommendations.

## Business Understanding
#### Problem statement
SyriaTel wants to understand and reduce customer churn. The company needs a way to identify customers who are likely to leave so that appropriate interventions can be applied (e.g. loyalty programs, special offers).
#### Stakeholders:
- Retention and Marketing Teams: need precise predictions to launch targeted retention campaigns.

- Executives: seek KPIs around churn mitigation and revenue protection.
#### Success Criteria: 
- High recall and precision for identifying churners.
- Interpretability to ensure actionable strategies.
- Practical business recommendations derived from model output.

## Data Understanding
The dataset includes 3,333 customers with features such as:

- Demographics: state, area code

- Service details: international plan, voice mail plan

- Usage metrics: daily, evening, night, and international minutes, calls, and charges

- Support history: number of customer service calls

- Target variable: churn (binary)

#### Initial Observations:

- Class imbalance: Majority class is non-churners (~86%)

- Potential multicollinearity: minutes and charges are highly correlated

- Some features (e.g. phone number) are identifiers and not predictive



## Data Preparation
#### Steps Taken:

1. Encoding:

- Binary features like international plan and voice mail plan are encoded into 0/1.

- The state column is label-encoded to numeric form.

2. Cleaning:

- Dropped phone number as it has no predictive value.

- Checked and confirmed no missing values.

3. Feature Engineering & Multicollinearity Check:

- Decided to retain either minutes or charges columns (not both).

- Considered aggregating call counts and charges into summaries.

4. Class Imbalance Handling:

- Used SMOTE to balance the training dataset.

5. Scaling:

- Applied StandardScaler for algorithms sensitive to magnitude (e.g., Logistic Regression, SVM).

## Modeling
Models used were selected for both performance and interpretability:

1. ### Baseline Model
##### Logistic Regression:

- Simple, interpretable.

- Trained on balanced and scaled data.

- Regularization used to avoid overfitting.

2. ### Advanced Model
##### Decision Tree Classifier:

- Captures non-linear patterns.

- Depth tuning applied for generalization.

3. ### Ensemble Model
##### Random Forest:

- Robust to overfitting, handles feature interactions.

- Used 100 trees and max depth tuning.

Optional models (not implemented):

##### k-Nearest Neighbors: 
- Would require further scaling and tuning.

##### SVM:
- Could be used, but computationally heavier.

## Evaluation
We chose metrics based on SyriaTel’s business needs:

- Recall: prioritized due to the high cost of missing a churner.

- Precision: useful to avoid wasting retention budget.

- F1 Score: used to balance both.

- ROC-AUC: shows model performance across thresholds.

All metrics were computed on both training and test sets. ROC curves were plotted for comparison.



## Conclusion
- Logistic Regression offered strong baseline performance and is easy to explain.

- Decision Tree exposed key features like customer service calls and international plan.

- Random Forest delivered higher accuracy and AUC by capturing more complex patterns.

- Feature importance plots revealed top drivers of churn.



## Recommendations
- Retention Strategy: Target users with frequent service calls and those with an international plan.

- Customer Experience: Investigate and resolve issues flagged in service calls.

- Marketing Offers: Design loyalty programs aimed at high-usage users with churn indicators.

- Model Deployment: Integrate the best model into CRM systems for real-time churn prediction.

- Monitoring: Continuously monitor model drift and retrain quarterly.

